# Estimating the impact of Government measures on COVID-19 infection spread through mobility data

We aim at answering the question :

**How does is the implementation of different strategies affecting the rates of COVID-19 infection ?**

## SEIR-HCD Model

Model built on compartmental models. Each of the following letters represents a compartment of the population of a country:

S - susceptible  
E - exposed  
I - infectious  
R - recovered  
H - hospitalized  
C - critical  
D - deceased  

We normalize these count numbers by the total population of the country.

In [8]:
%%latex
\begin{align}
\frac{dS}{dt} &= - \frac{R_t}{T_{inf}} I S \\
\frac{dE}{dt} &= \frac{R_t}{T_{inf}} I S - \frac{E}{T_{inc}} \\
\frac{dI}{dt} &= \frac{E}{T_{inc}} - \frac{I}{T_{inf}} \\
\frac{dR}{dt} &= m_a \frac{I}{T_{inf}} + (1 - c_a)\frac{H}{T_{hosp}} \\
\frac{dH}{dt} &= (1 - m_a) \frac{I}{T_{inf}} + (1 - f_a)\frac{C}{T_{crit}} - \frac{H}{T_{hosp}} \\
\frac{dC}{dt} &= c_a \frac{H}{T_{hosp}} - \frac{C}{T_{crit}} \\
\frac{dD}{dt} &= f_a \frac{C}{T_{crit}}
\end{align}

<IPython.core.display.Latex object>

### Parameters used in the model
$R_t$ = reproduction number at time t.  
Typical 3.6* at t=0

Transition times

    T_inc = average incubation period. Typical 5.6* days
    T_inf = average infectious period. Typical 2.9 days
    T_hosp = average time a patient is in hospital before either recovering or becoming critical. Typical 4 days
    T_crit = average time a patient is in a critical state (either recover or die). Typical 14 days

Fractions  
These constants are likely to be age specific (hence the subscript a):

    m_a = fraction of infections that are asymptomatic or mild. Assumed 80% (i.e. 20% severe)
    c_a = fraction of severe cases that turn critical. Assumed 10%
    f_a = fraction of critical cases that are fatal. Assumed 30%
    
*Averages taken from https://www.kaggle.com/covid-19-contributions

### Reproduction number and mobility
let $\mathrm{m}_t$ be the reduction of mobility relatively to a baseline (before the pandemic).  
Hence $\mathrm{m}_t > -1$.

We assume the following relationship:
$$ R_t = (1 + \mathrm{m}_t)R_0 - \mathrm{m}_tR_1  $$
for any time $t$ through the pandemic, with $R_0$, $R_1$ to be estimated.

## Bayesian modelling

We use the numpyro probabilistic programming framework.

We use the following parameterizations:
- (concentration, rate) for the $\mathrm{GammaPoisson}$ distribution.
- (mean, std) for the $\mathrm{Gamma}$ distribution
- (mean, sample size) for the $\mathrm{Beta}$ distribution

### Target
Let $D_t$ be the number of death at time $t$ for a given country. We model $d_t = \mathbb{E}[D_t]$

We sample 
$$D_t \sim \mathrm{GammaPoisson}(\psi d_t, \psi)$$
where $\psi \sim \mathcal{N}^+(0, 5)$

### Compartment init
We seed the model with the following, assuming $N$ is the country population:
- $I_0 \sim Gamma(100, 50) / N$
- $S_0 = 1 - I_0$
- $E_0, R_0, H_0, C_0, D_0 = 0$

### Other priors
$R_0 \sim \mathcal{N}^+(3.28, \kappa_0)$

$R_1 \sim \mathcal{N}^+(0.7, \kappa_1)$

where $\kappa_0, \kappa_1 \sim \mathcal{N}^+(0, .5)$


$T_{inc} \sim \mathrm{Gamma}(5.6, .86)$

$T_{inc} \sim \mathrm{Gamma}(2.9, 1)$

$T_{hosp} \sim \mathrm{Gamma}(4, 1)$

$T_{crit} \sim \mathrm{Gamma}(14, 1)$

$m_a \sim \mathrm{Beta}(0.8, \phi)$

$c_a \sim \mathrm{Beta}(0.1, \phi)$

$f_a \sim \mathrm{Beta}(0.35, \phi)$

where $\phi \sim \mathrm{Exponential}(1)$


# Sources


[Prior work](https://github.com/horaceg/uncover/blob/master/working/Measures%20and%20rate.html) (to be downloaded):  
- Map + time series of measures with cases and mobility
- Minimization of parameters for the compartment model

*Sources*

- Imperial team
    - [Report 13](https://www.imperial.ac.uk/mrc-global-infectious-disease-analysis/covid-19/report-13-europe-npi-impact/): 30 March 2020
    - [Technical update](https://arxiv.org/abs/2004.11342): 23 april 2020
    - [Code](https://github.com/ImperialCollegeLondon/covid19model) in R and Stan
    - [website](https://mrc-ide.github.io/covid19estimates/#/)


- Compartment models
    - [Wikipedia](https://en.wikipedia.org/wiki/Compartmental_models_in_epidemiology)
    - SEIR-HCD [Kaggle kernel](https://www.kaggle.com/anjum48/seir-hcd-model) from @anjum (Datasaurus). Sources of inspiration:
        - [Neher labs](https://covid19-scenarios.org/) 
        - [Gabriel Goh](https://gabgoh.github.io/COVID/index.html) 


- Probabilistic programming
    - [numpyro](https://github.com/pyro-ppl/numpyro) ([paper](https://arxiv.org/abs/1912.11554)): probabilistic programming library in python, built on [JAX](https://github.com/google/jax)
    - [SEIR ode modelling](https://docs.pymc.io/notebooks/ODE_API_introduction.html) in Pymc3 
    - [Predator-Prey ode](http://pyro.ai/numpyro/examples/ode.html) in Numpyro 
    - Another [predator-prey](https://fehiepsi.github.io/rethinking-numpyro/16-generalized-linear-madness.html) ode in numpyro
    - [A bayesian SIR](https://github.com/twiecki/covid19) modelling with Pymc3 with the [video](https://youtu.be/C1kWBTj6KvE?t=410)
    - [Estimating COVID-19's $R_t$ in Real-Time with PYMC3](https://github.com/k-sys/covid-19/blob/master/Realtime%20Rt%20mcmc.ipynb)

- Misc
    - [generation interval](https://www.medrxiv.org/content/10.1101/2020.03.05.20031815v1)


*Data sources*

- Epidemiology
    - [Our world in data](https://github.com/owid/covid-19-data/tree/master/public/data): confirmed cases and deaths from ECDC (European center of Disease Control) + tests (collected by owid)


- Mobility
    - [Apple](https://www.apple.com/covid19/mobility) - 3 categories: walking, driving, transit
    - [Google](https://www.google.com/covid19/mobility/) - 6 categories


- Government Measures
    - [ACAPS](https://data.humdata.org/dataset/acaps-covid19-government-measures-dataset)
    - [Oxford](https://www.bsg.ox.ac.uk/research/research-projects/coronavirus-government-response-tracker)
